# 67373视频推荐系统

In [1]:
# 从本地读取数据集
f = open("dataset/dataset.txt", "r")
lines = f.read().splitlines()
dataset = []
for line in lines:
    # arr格式为: [title, cate_name, cate_id, duration, creation_time, video_id]
    arr = line.split(',')
    dataset.append(arr)

## 创建codebook

In [2]:
# 提取数据集每行的前两个元素（即title和cate_name），以字为单位创建codebook
codebook = set()
for data in dataset:
    codebook.update(data[0], data[1])
codebook = list(codebook)
print('codebook长度: {}个字'.format(len(codebook)))
print('codebook内容:\n{}'.format(codebook))

codebook长度: 671个字
codebook内容:
['打', '0', '忘', '从', '其', '表', '聚', '受', '冰', '形', '格', 'S', '公', '通', '内', '是', '暗', 'c', '还', 'Q', '她', '由', '加', '逆', ']', '美', '鸟', '特', '男', '演', '外', '良', '往', '必', '蛋', '勒', '烟', '爱', '质', 'F', '谢', '咪', '校', '县', 'N', '了', '冬', '杂', '鸡', '愁', 's', '验', '宝', '红', '鬼', '运', '多', '丁', '弦', '米', '全', '前', '斗', '宇', '要', '专', 'P', '翻', '泪', '首', '金', '聊', '赚', '白', '播', '醉', '衣', '欣', '老', '病', '到', '快', 'B', '意', '影', 'R', '面', '隐', '目', '种', '罗', '每', '゙', '夸', '几', '就', '朝', '题', '孩', 'i', '2', '亲', '远', '刻', '而', '管', '让', '歌', '眼', '倾', '寻', '馆', '“', 'て', '色', '长', '徒', '买', '姑', '何', '安', '马', '乎', '一', '友', 'e', '参', '极', '伴', '仔', '识', 'U', '错', '送', '各', '价', '东', '1', '邀', '也', '故', '着', '。', '阙', '卡', '澡', '岁', '曲', '喜', 'O', '兰', '忧', '插', '感', '两', '年', '走', '夜', '划', '利', '导', '亮', 'Y', '盼', '新', '绝', '鲸', '陌', '少', '点', '写', '行', '克', '漫', '劝', '凡', '站', 'g', '们', '诱', '这', '花', '核', '人', '宗', '完', '约', 'w', '总', '足', '为', '骑', '力', '吴',